In [ ]:
import requests
import pandas as pd
from datetime import datetime
import os

# 1. 定义目标 URL (Reddit 新加坡板块)
url = "https://www.reddit.com/r/singapore/hot/.json?limit=100"

# 2. 设置 User-Agent (必须设置，否则会被 Reddit 拒绝访问)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}



In [2]:
def clean_text(text):
    import html
    import re
    if not text or not isinstance(text, str):
        return ""
    
    # 1. 还原 HTML 实体
    text = html.unescape(text)
    
    # 2. 核心修改：显式删除所有形式的换行与回车
    # \r 是回车，\n 是换行
    text = text.replace('\r', ' ').replace('\n', ' ')
    
    # 3. 删除引号及反斜杠
    quotes_pattern = r'["\'“”‘’\'\\]' 
    text = re.sub(quotes_pattern, '', text)
    
    # 4. 映射其他特殊符号
    punctuation_map = {
        '—': '-', 
        '\xa0': ' ',
        '\u200b': '' # 零宽空格
    }
    text = text.translate(str.maketrans(punctuation_map))
    
    # 5. 深度清理：将所有连续的空白符（空格/制表符等）合并为一个空格
    # 这步非常重要，能确保删掉换行后不会留下奇怪的间距
    text = " ".join(text.split())
    
    return text.strip()

In [ ]:
try:
    # 3. 发送请求
    response = requests.get(url, headers=headers)
    response.raise_for_status() # 如果请求失败则抛出异常
    
    # 4. 解析 JSON 数据
    raw_data = response.json()
    posts = raw_data['data']['children']
    
    extracted_data = []
    
    for post in posts:
        data = post['data']
        extracted_data.append({
            'post_id': data.get('id'),
            'title': clean_text(data.get('title')),
            'author': clean_text(data.get('author')),
            'score': data.get('score'),       # 点赞数
            'upvote_ratio': data.get('upvote_ratio'),
            'comments': data.get('num_comments'),
            'flair': data.get('link_flair_text'), # 帖子标签 (如 News, AskSingapore)
            'is_video': data.get('is_video'),
            'is_self': data.get('is_self'),
            'domain' : data.get('domain'),
            'url': data.get('url'),
            'created_utc': datetime.fromtimestamp(data.get('created_utc')).strftime('%Y-%m-%d %H:%M:%S'),
            'selftext': clean_text(data.get('selftext')),
            'extracted_time' : datetime.now()
        })

    # 5. 转换为 DataFrame
    df = pd.DataFrame(extracted_data)

    # 定义基础路径 (对应 Astro 容器内的位置)
    base_path = "/usr/local/airflow/include/data"

    # 自动检查目录是否存在，不存在则创建
    if not os.path.exists(base_path):
        os.makedirs(base_path)

    file_name = os.path.join(base_path, f"reddit_sg_posts_{datetime.now().strftime('%Y%m%d_%H%M')}.csv")
    df.to_csv(file_name, index=False, encoding='utf-8-sig') # utf-8-sig 确保 Excel 打开不乱码

    print(f"成功抓取 {len(df)} 条帖子，已保存至: {file_name}")
    print(df[['title', 'score']].head()) # 打印前几行预览

except Exception as e:
    print(f"抓取失败: {e}")

成功抓取 25 条帖子，已保存至: data/reddit_sg_posts_20260108_1346.csv
                                               title  score
0  r/singapore random discussion and small questi...     13
1  Woman arrested at New Zealand airport after 18...    214
2  Steven Lim Overshares About Girlfriends Past I...    144
3  More than 57,000 Malaysians gave up citizenshi...     86
4  Its a contravention of international law: SM L...     39
